In [ ]:
# Author: Ryan Ku

import pandas as pd
import numpy as np
from datetime import date

In [ ]:
# thank you @rogerallen: https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
# returns a partial dataframe filtered by col:text (key:value) pair in filter_dict (i.e. removes all rows that don't satisfy filter_dict)
# if remove_dict is specified, this function removes rows that satisfy remove_dict, BUT keeps any that also satisfies filter_dict
def filter_by_value(df, filter_dict: dict = {}, remove_dict: dict = {}):
    # if remove_dict was specified, remove rows that satisfy remove_dict BUT not filter_dict:
    i_remove = set()
    i_keep = set()
    if len(remove_dict) > 0:
        for col, text in remove_dict.items():
            i_remove = i_remove.union(set(df[df[col].str.contains(text)].index))
        for col, text in filter_dict.items():
            i_keep = i_keep.union(set(df[df[col].str.contains(text)].index))
    # if remove_dict was not specified, filter rows by filter_dict:
    else:
        for col, text in filter_dict.items():
            i_remove = i_remove.union(set(df[~df[col].str.contains(text)].index))
        
    return df.drop(index = i_remove - i_keep)

# removes all rows that don't correspond to individual counties (only works on USDoA datasets)
# NOTE: boroughs are ignored for sake of simplicity (there's only ~15-20 anyways)
def filter_counties(df):
    i_remove = df[~df["County"].str.contains("County")].index   # remove rows that don't contain the word "County" in the "County" column
    return df.drop(index = i_remove)

# return true if NAs found under specific column
def any_na(df, col) -> bool:
    return df[col].isnull().values.any()

In [ ]:
# retrieves relevant virus data
def get_virus_data():
    dtypes = {
        "fips" : "Int32",
        "cases" : "Int32",
        "deaths" : "Int32"
    }
    columns = {
        "date" : "Date",
        "county" : "County",
        "state" : "State",
        "fips" : "FIPS",
        "cases" : "Cases",
        "deaths" : "Deaths"
    }
    data = pd.read_csv("./datasets/NYTimes/us-counties_2020_5_21.csv", sep=",", header=0, engine="python", dtype=dtypes, parse_dates=["date"]).rename(columns=columns)

    # remove data from unknown locations, BUT keep Puerto Rico data
    data = filter_by_value(data, filter_dict={"State": "Puerto Rico"}, remove_dict={"County": "Unknown"})
    # add Days_since_first column to track number of days since start of data
    data["Days_since_first"] = (data["Date"] - data["Date"][0]).dt.days
    # TODO: convert state to state abbreviation
    return data

# retrieves relevant education data
def get_education_data():
    dtype = {
        "FIPS Code" : "Int32",
        "Less than a high school diploma, 2014-18" : "Int32",
        "High school diploma only, 2014-18" : "Int32",
        "Some college or associate's degree, 2014-18" : "Int32",
        "Bachelor's degree or higher, 2014-18" : "Int32",
    }
    columns = {
        "FIPS Code" : "FIPS",
        "State" : "State",
        "Area name" : "County",
        "Less than a high school diploma, 2014-18" : "Less_HS",
        "High school diploma only, 2014-18" : "HS_only",
        "Some college or associate's degree, 2014-18" : "College_Associate",
        "Bachelor's degree or higher, 2014-18" : "Bachelors",
        "Percent of adults with less than a high school diploma, 2014-18" : "PCT_Less_HS",
        "Percent of adults with a high school diploma only, 2014-18" : "PCT_HS_only",
        "Percent of adults completing some college or associate's degree, 2014-18" : "PCT_College_Associate",
        "Percent of adults with a bachelor's degree or higher, 2014-18" : "PCT_Bachelors"
    }
    data = pd.read_csv("./datasets/USDoA/Education_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant population data
def get_population_data():
    dtype = {
        "FIPS" : "Int32",
        "CENSUS_2010_POP" : "Int32",
        "POP_ESTIMATE_2018" : "Int32",
        "Births_2018" : "Int32",
        "Deaths_2018" : "Int32",
        "INTERNATIONAL_MIG_2018" : "Int32",
        "DOMESTIC_MIG_2018" : "Int32",
        "NET_MIG_2018" : "Int32",
        "GQ_ESTIMATES_2018" : "Int32"
    }
    columns = {
        "FIPS" : "FIPS",
        "State" : "State",
        "Area_Name" : "County",
        "CENSUS_2010_POP" : "CENSUS_2010_POP",                     # 4/1/2010 resident total Census 2010 population
        "POP_ESTIMATE_2018" : "POP_ESTIMATE_2018",                 # 7/1/2018 resident total population estimate
        "Births_2018" : "Births_2018",                             # Births in period 7/1/2017 to 6/30/2018
        "Deaths_2018" : "Deaths_2018",                             # Deaths in period 7/1/2017 to 6/30/2018
        "INTERNATIONAL_MIG_2018" : "INTERNATIONAL_MIG_2018",       # Net international migration in period 7/1/2017 to 6/30/2018
        "DOMESTIC_MIG_2018" : "DOMESTIC_MIG_2018",                 # Net domestic migration in period 7/1/2017 to 6/30/2018
        "NET_MIG_2018" : "NET_MIG_2018",                           # Net migration in period 7/1/2017 to 6/30/2018
        "GQ_ESTIMATES_2018" : "GQ_ESTIMATES_2018",                 # 7/1/2018 Group Quarters total population estimate
        "R_birth_2018" : "R_birth_2018",                           # Birth rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_death_2018" : "R_death_2018",                           # Death rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_INTERNATIONAL_MIG_2018" : "R_INTERNATIONAL_MIG_2018",   # Net international migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_DOMESTIC_MIG_2018" : "R_DOMESTIC_MIG_2018",             # Net domestic migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
        "R_NET_MIG_2018" : "R_NET_MIG_2018"                        # Net migration rate in period 7/1/2017 to 6/30/2018 (per thousand people?)
    }
    data = pd.read_csv("./datasets/USDoA/PopulationEstimates_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant poverty data
def get_poverty_data():
    dtype = {
        "FIPStxt" : "Int32",
        "POVALL_2018" : "Int32",
        "POV017_2018" : "Int32",
        "POV517_2018" : "Int32",
        "MEDHHINC_2018" : "Int32"
    }
    columns = {
        "FIPStxt" : "FIPS",
        "Stabr" : "State",
        "Area_name" : "County",
        "POVALL_2018" : "Poverty_Total",               # total amount in poverty in 2018
        "PCTPOVALL_2018" : "PCT_Poverty",              # percentage in poverty in 2018
        "POV017_2018" : "Poverty_0_17",                # amount in poverty ages 0-17 in 2018
        "PCTPOV017_2018" : "PCT_Poverty_0_17",         # percentage in poverty ages 0-17 in 2018
        "POV517_2018" : "Poverty_5_17",                # amount in poverty ages 5-17 in 2018
        "PCTPOV517_2018" : "PCT_Poverty_5_17",         # percentage in poverty ages 5-17 in 2018
        "MEDHHINC_2018" : "Median_Household_Income"    # median household income in 2018
    }
    data = pd.read_csv("./datasets/USDoA/PovertyEstimates_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    return filter_counties(data)

# retrieves relevant unemployment data
def get_unemployment_data():
    dtype = {
        "FIPS" : "Int32",
        "Civilian_labor_force_2018" : "Int32",
        "Employed_2018" : "Int32",
        "Unemployed_2018" : "Int32",
    }
    columns = {
        "FIPS" : "FIPS",
        "State" : "State",
        "Area_name" : "County",
        "Civilian_labor_force_2018" : "Total_Labor_Force",
        "Employed_2018" : "Employed",
        "Unemployed_2018" : "Unemployed",
        "Unemployment_rate_2018" : "PCT_Unemployed",
        "Median_Household_Income_2018" : "Median_Household_Income",
        "Med_HH_Income_Percent_of_State_Total_2018" : "Median_Household_Income_County_to_State_Ratio"
    }
    data = pd.read_csv("./datasets/USDoA/Unemployment_2020_5_7.csv", sep=",", header=0, engine="python", usecols=columns.keys(), dtype=dtype, thousands=",").rename(columns=columns)
    # remove non-counties
    data = filter_counties(data)
    # strip and convert "Median_Household_Income" column to int
    data["Median_Household_Income"] = data["Median_Household_Income"].astype(str).map(lambda x: x.lstrip("$").rstrip(" ").replace(",", "")).astype(int)
    return data
    
# retrieves relevant land area data
def get_land_data():
    columns = {
        "Areaname" : "Areaname",      # County, State
        "STCOU" : "FIPS",
        "LND110210D" : "LANDAREA",    # -> Land area in square miles 2010
        "LND210200D" : "WATERAREA"    # -> Water area in square miles 2000
    }
    data = pd.read_csv("./datasets/USCensus/LND01_2020_5_21.csv", sep=",", header=0, engine="python", usecols=columns.keys()).rename(columns=columns)
    
    # split county and state names into separate columns
    new = data["Areaname"].str.split(pat=", ", n=1, expand=True)
    data["County"] = new[0] + " County"
    data["State"] = new[1]
    data.drop(columns=["Areaname"], inplace=True)
    
    # remove state rows (i.e. rows that describe the entire state)
    # aka rows where the "State" column is now null due to split()
    data.dropna(axis="index", inplace=True)
    
    return data

In [ ]:
virus_data = get_virus_data()
education_data = get_education_data()
population_data = get_population_data()
poverty_data = get_poverty_data()
unemployment_data = get_unemployment_data()
land_data = get_land_data()

# NOTE: As far as I can tell, the COVID19 data from NYTimes seems to consolidate 5 counties under New York City (and thus it has no listed FIPS code)
#       as a result, I will have to simply link the 5 counties (Manhattan/New York, Queens, Bronx, Brooklyn/Kings, Staten Island/Richmond).
#       For more information: https://github.com/nytimes/covid-19-data/issues/105
# get FIPS codes corresponding to the 5 NYC counties
nyc_FIPS = list()
nyc_FIPS.append(filter_by_value(population_data, {"State": "NY", "County": "Bronx County"}).iloc[0]["FIPS"])
nyc_FIPS.append(filter_by_value(population_data, {"State": "NY", "County": "Kings County"}).iloc[0]["FIPS"])
nyc_FIPS.append(filter_by_value(population_data, {"State": "NY", "County": "New York County"}).iloc[0]["FIPS"])
nyc_FIPS.append(filter_by_value(population_data, {"State": "NY", "County": "Queens County"}).iloc[0]["FIPS"])
nyc_FIPS.append(filter_by_value(population_data, {"State": "NY", "County": "Richmond County"}).iloc[0]["FIPS"])
print(nyc_FIPS)

In [ ]:
virus_data
# print(filter_by_value(virus_data, {"state": "New York"}))

In [ ]:
education_data.dtypes
# any_na(education_data, "FIPS")
# unemployment_data.iloc[92]
# len(unemployment_data[unemployment_data.isna().any(axis=1)])